### Notebook for testing out the various functions needed to work with the arxiv metadata and extract relevent features such as authors, citations, etc., from it and push into a feature store

TO DO
1. Figure out how to ensure that the arxiv IDs here are the same as from citations functions
2. How do you get the year of an arxiv publication?
3. Need to get proper catalog of all the categories

In [3]:
import os
import sys
sys.path.append(os.path.abspath("/home/arxiv/doc_intel_etl"))
import config
from dataclasses import dataclass
import gzip
import glob
import json
import time
import hashlib
import datetime
import requests
import json
import torch
import ray
import xml.etree.ElementTree as ET
import re
os.environ['PYTHONPATH'] = os.path.dirname(os.getcwd())
# from src.get_metadata import publication
import multiprocessing as mp
import pandas as pd
from src.authors import parse_authorline

In [4]:
METADATA_FILE_LOC = config.metadata_file_loc
EMBED_MODEL_API = config.embed_model_api

In [5]:
ray.init()

2020-08-08 03:56:00,365	INFO resource_spec.py:212 -- Starting Ray with 37.35 GiB memory available for workers and up to 18.68 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-08 03:56:00,733	INFO services.py:1170 -- View the Ray dashboard at localhost:8265
2020-08-08 03:56:00,738	WARNING services.py:1494 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.


{'node_ip_address': '172.17.0.2',
 'raylet_ip_address': '172.17.0.2',
 'redis_address': '172.17.0.2:14171',
 'object_store_address': '/tmp/ray/session_2020-08-08_03-56-00_363079_3066/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-08-08_03-56-00_363079_3066/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-08-08_03-56-00_363079_3066'}

2020-08-08 03:56:01,721	WARNING worker.py:1090 -- The dashboard on node bc572e8fa404 failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/ray/dashboard/dashboard.py", line 1220, in <module>
    dashboard.run()
  File "/opt/conda/lib/python3.7/site-packages/ray/dashboard/dashboard.py", line 594, in run
    aiohttp.web.run_app(self.app, host=self.host, port=self.port)
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web.py", line 433, in run_app
    reuse_port=reuse_port))
  File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 587, in run_until_complete
    return future.result()
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web.py", line 359, in _run_app
    await site.start()
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_runner.py", line 104, in start
    reuse_port=self._reuse_port)
  File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 1389, in create_server
    % (sa, err.strerr

In [94]:
def load_metadata(fname):
    """
    Load metadata saved by all_of_arxiv, as a list of lines of gzip
    compressed json.

    Parameters
    ----------
        infile : str or None
            name of file saved by gzip. If None, one is attempted to be
            found in the expected location with the expected name.

    Returns
    -------
        article_attributes : list
            list of dicts, each of which contains the metadata
            attributes of the ArXiv articles
    """
    with gzip.open(fname, 'rt', encoding='utf-8') as fin:
        return [json.loads(line) for line in fin.readlines()]

def split_metadata(metadata, cpu_count):
    '''
    Split the large metadata file into chunks
    '''
    chunk_size = len(metadata)//cpu_count
    print("Chunk Size: ", chunk_size)
    chunked_metadata = []
    for i in range(0, len(metadata), chunk_size):
        if i+chunk_size <= len(metadata):
            chunked_metadata.append(metadata[i:i+chunk_size])
        else:
            chunked_metadata.append(metadata[i:])
    return chunked_metadata

@ray.remote
def parse_author_line(authors):
    return parse_authorline(authors).split('; ')

def get_author_df(authors):
    authors = ray.get([parse_author_line.remote(article)
                       for article in authors])
    authors = [author for article in authors
                  for author in article]
    authors = list(set(authors))
    df_authors = pd.DataFrame(authors, columns=['author_name'])
    df_authors['authorId:ID(Author-ID)'] = df_authors.reset_index()['index']
    df_authors[':LABEL'] = 'author'
    return df_authors

def parse_categories(cat_list):
    return cat_list[0].strip(' ')

def parse_abstract(abstract):
    return abstract.replace('\n','')[2:]

def get_article_df(articles):
    df_articles = pd.DataFrame(columns=articles_cols)
    df_articles['title'] = articles['title']
#     self.df_articles['pub_date'] = self.get_pub_date(articles['pub_id'])
    df_articles['categories'] = articles['categories'].apply(parse_categories)
    df_articles['report-no'] = articles['report-no']
    # all the abstracts have a space in front so the [2:] just
    # removes that space
    df_articles['abstract'] = articles['abstract'].apply(parse_abstract)
    df_articles['doi'] = articles['doi']
    df_articles['articleId:ID(Article-ID)'] = articles['id']
    return df_articles

In [7]:
metadata = load_metadata(METADATA_FILE_LOC)

In [9]:
sample_df = pd.DataFrame.from_dict(metadata, orient='columns')

In [48]:
articles_cols = ['articleId:ID(Article-ID)',
                         'title',
                         'pub_date',
                         'categories',
                         'report-no',
                         'abstract',
                         'doi']
rels_cols = [':START_ID(Author-ID)',':END_ID(Article-ID)',':TYPE']

In [95]:
article_df = get_article_df(sample_df[:10])
article_df

,articleId:ID(Article-ID),title,pub_date,categories,report-no,abstract,doi
0,0704.0001,Calculation of prompt diphoton production cros...,NaN,hep-ph,ANL-HEP-PR-07-12,A fully differential calculation in perturbati...,10.1103/PhysRevD.76.013009
1,0704.0002,Sparsity-certifying Graph Decompositions,NaN,math.CO cs.CG,None,"We describe a new algorithm, the $(k,\ell)$-pe...",None
2,0704.0003,The evolution of the Earth-Moon system based o...,NaN,physics.gen-ph,None,The evolution of Earth-Moon system is describe...,None
3,0704.0004,A determinant of Stirling cycle numbers counts...,NaN,math.CO,None,We show that a determinant of Stirling cycle n...,None
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,NaN,math.CA math.FA,None,In this paper we show how to compute the $\Lam...,None
5,0704.0006,Bosonic characters of atomic Cooper pairs acro...,NaN,cond-mat.mes-hall,None,We study the two-particle wave function of pai...,10.1103/PhysRevA.75.043613
6,0704.0007,Polymer Quantum Mechanics and its Continuum Limit,NaN,gr-qc,IGPG-07/03-2,A rather non-standard quantum representation o...,10.1103/PhysRevD.76.044016
7,0704.0008,Numerical solution of shock and ramp compressi...,NaN,cond-mat.mtrl-sci,"LA-UR-07-2051, LLNL-JRNL-410358",A general formulation was developed to represe...,10.1063/1.2975338
8,0704.0009,"The Spitzer c2d Survey of Large, Nearby, Inste...",NaN,astro-ph,None,We discuss the results from the combined IRAC ...,10.1086/518646
9,0704.0010,"Partial cubes: structures, characterizations, ...",NaN,math.CO,None,Partial cubes are isometric subgraphs of hyper...,None


In [82]:
sample_df.abstract[0].replace('\n','')[2:]

'A fully differential calculation in perturbative quantum chromodynamics ispresented for the production of massive photon pairs at hadron colliders. Allnext-to-leading order perturbative contributions from quark-antiquark,gluon-(anti)quark, and gluon-gluon subprocesses are included, as well asall-orders resummation of initial-state gluon radiation valid atnext-to-next-to-leading logarithmic accuracy. The region of phase space isspecified in which the calculation is most reliable. Good agreement isdemonstrated with data from the Fermilab Tevatron, and predictions are made formore detailed tests with CDF and DO data. Predictions are shown fordistributions of diphoton pairs produced at the energy of the Large HadronCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgsboson are contrasted with those produced from QCD processes at the LHC, showingthat enhanced sensitivity to the signal can be obtained with judiciousselection of events.'

In [117]:
sample_df

,id,submitter,authors,title,comments,journal-ref,doi,abstract,report-no,categories,versions
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,A fully differential calculation in perturba...,ANL-HEP-PR-07-12,[hep-ph],"[v1, v2]"
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,"We describe a new algorithm, the $(k,\ell)$-...",None,[math.CO cs.CG],"[v1, v2]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,The evolution of Earth-Moon system is descri...,None,[physics.gen-ph],"[v1, v2, v3]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,We show that a determinant of Stirling cycle...,None,[math.CO],[v1]
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,In this paper we show how to compute the $\L...,None,[math.CA math.FA],[v1]
...,...,...,...,...,...,...,...,...,...,...,...
1645522,supr-con/9608008,Ruslan Prozorov,"R. Prozorov, M. Konczykowski, B. Schmidt, Y. Y...",On the origin of the irreversibility line in t...,"19 pages, LaTex, 6 PostScript figures; Author'...",None,10.1103/PhysRevB.54.15530,We report on measurements of the angular dep...,None,[supr-con cond-mat.supr-con],[v1]
1645523,supr-con/9609001,Durga P. Choudhury,"Durga P. Choudhury, Balam A. Willemsen, John S...",Nonlinear Response of HTSC Thin Film Microwave...,"4 pages, LaTeX type, Uses IEEE style files, 60...",None,10.1109/77.620744,The non-linear microwave surface impedance o...,None,[supr-con cond-mat.supr-con],[v1]
1645524,supr-con/9609002,Durga P. Choudhury,"Balam A. Willemsen, J. S. Derov and S.Sridhar ...",Critical State Flux Penetration and Linear Mic...,"20 pages, LaTeX type, Uses REVTeX style files,...",None,10.1103/PhysRevB.56.11989,The vortex contribution to the dc field (H) ...,None,[supr-con cond-mat.supr-con],[v1]
1645525,supr-con/9609003,Hasegawa Yasumasa,Yasumasa Hasegawa (Himeji Institute of Technol...,Density of States and NMR Relaxation Rate in A...,"7 pages, 4 PostScript Figures, LaTeX, to appea...",None,10.1143/JPSJ.65.3131,We show that the density of states in an ani...,None,[supr-con cond-mat.supr-con],[v1]


In [113]:
author_list = [author for article in authors_series.str.split('; ')
              for author in article]
# remove duplicates
author_list = list(set(author_list))
author_list = pd.Series(author_list)
# author_list = author_list.str.replace(' ','')
author_list

0                       Xu, Xiao-bao
1                       Guyot, Cyril
2          Aschenauer, Elke-Caroline
3                      Sharma, Arjun
4               Breitsprecher, Thilo
                     ...            
1043958       Bhattacharyya, Rangeet
1043959            Silva, M. Zimbres
1043960                McShane, Greg
1043961                  Retzker, A.
1043962               Xiao, Weiliang
Length: 1043963, dtype: object

## Test out Pytorch embedding of abstract with huggingface API

In [19]:
%%timeit
model_loc = 'http://localhost:5001/embed'
abstract = get_abstract(metadata[54])
payload = create_text_payload(abstract)
embedding = get_embedding(model_loc, payload)
print(embedding.size())

torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size

## Test out building pytorch model with torchserve

In [26]:
from transformers import DistilBertModel, DistilBertTokenizer
path = '/home/arxiv/doc_intel_etl/models/text_embedding/distilbert'
# save a pytorch model and tokenizer
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model.eval()
# since it's a hugging face pre-trained model need to use save_pretrained
model.save_pretrained(path)
tokenizer.save_pretrained(path)

('/home/arxiv/doc_intel_etl/models/text_embedding/distilbert/vocab.txt',
 '/home/arxiv/doc_intel_etl/models/text_embedding/distilbert/special_tokens_map.json',
 '/home/arxiv/doc_intel_etl/models/text_embedding/distilbert/added_tokens.json')

In [43]:
get_pub_date(metadata[100000]['id'])

('2008', '12')

## Test citations file

In [54]:
file = '/home/arxiv/doc_intel_etl/internal-citations.json.gz'
cite = []
with gzip.open(file, 'rt') as f:
    for line in f:
        cite.append(line)

In [59]:
cite[0]

'[{"hep-lat/9107001": []}, {"hep-lat/9107002": []}, {"hep-lat/9112001": []}, {"hep-lat/9112002": []}, {"hep-th/9108001": []}, {"hep-th/9108002": []}, {"hep-th/9108003": []}, {"hep-th/9108004": []}, {"hep-th/9108005": []}, {"hep-th/9108006": []}, {"hep-th/9108007": []}, {"hep-th/9108008": []}, {"hep-th/9108009": []}, {"hep-th/9108010": []}, {"hep-th/9108011": []}, {"hep-th/9108013": []}, {"hep-th/9108014": []}, {"hep-th/9108015": []}, {"hep-th/9108016": []}, {"hep-th/9108017": []}, {"hep-th/9108018": []}, {"hep-th/9108019": []}, {"hep-th/9108020": []}, {"hep-th/9108021": []}, {"hep-th/9108022": []}, {"hep-th/9108023": []}, {"hep-th/9108024": []}, {"hep-th/9108025": []}, {"hep-th/9108026": []}, {"hep-th/9108027": []}, {"hep-th/9108028": []}, {"hep-th/9109001": []}, {"hep-th/9109002": []}, {"hep-th/9109003": []}, {"hep-th/9109004": []}, {"hep-th/9109005": []}, {"hep-th/9109006": []}, {"hep-th/9109007": []}, {"hep-th/9109008": []}, {"hep-th/9109009": []}, {"hep-th/9109010": []}, {"hep-th/9

In [21]:
test

'['